# **Network Data Source**: EDA
------------------

In [1]:
# https://infosecjupyterthon.com/notebooks/2_Jupyterthon_Cyb3rPandaH_2020.html
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql.types import *
import pyspark.sql.functions as func
from pyspark.sql.functions import length, concat ,concat_ws

import pandas as pd, numpy as np, networkx as nx
import matplotlib.pyplot as plt
import requests
from openhunt import ossem, descriptiveStatistics as ds, visualizations as vis
import warnings
warnings.filterwarnings('ignore')

# Import packages
import os
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
%matplotlib inline

In [2]:
!wget https://raw.githubusercontent.com/OTRF/detection-hackathon-apt29/master/datasets/day1/zeek/combined_zeek.log

--2020-06-23 10:36:47--  https://raw.githubusercontent.com/OTRF/detection-hackathon-apt29/master/datasets/day1/zeek/combined_zeek.log
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.52.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1243861 (1.2M) [text/plain]
Saving to: ‘combined_zeek.log’

combined_zeek.log   100%[===================>]   1.19M  6.93MB/s    in 0.2s    

2020-06-23 10:36:48 (6.93 MB/s) - ‘combined_zeek.log’ saved [1243861/1243861]



In [11]:
# Reference: https://infosecjupyterthon.com/notebooks/2_Jupyterthon_Cyb3rPandaH_2020.html

In [16]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.caseSensitive", "true")
spark.sparkContext.setCheckpointDir("./tmp/checkpoints")
spark.conf.set("spark.driver.memory", "3g")
spark.conf.set("spark.executor.memory", "3g")
spark.sparkContext.version

'2.4.5'

In [17]:
zeekDf_day1 = pd.read_json("combined_zeek.log", lines = True)

print(type(zeekDf_day1))
zeekDf_day1.head()

<class 'pandas.core.frame.DataFrame'>


,@stream,@system,@proc,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,proto,...,is_64bit,uses_aslr,uses_dep,uses_code_integrity,uses_seh,has_import_table,has_export_table,has_cert_table,has_debug_data,section_names
0,conn,bobs.bigwheel.local,zeek,1.588205e+09,Cvf4XX17hSAgXDdGEd,10.0.1.6,54243.0,10.0.0.4,53.0,udp,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,conn,bobs.bigwheel.local,zeek,1.588205e+09,CJ21Le4zsTUcyKKi98,10.0.1.6,56880.0,10.0.0.4,445.0,tcp,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,conn,bobs.bigwheel.local,zeek,1.588205e+09,CnOP7t1eGGHf6LFfuk,10.0.1.6,65108.0,10.0.0.4,53.0,udp,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,conn,bobs.bigwheel.local,zeek,1.588205e+09,CvxbPE3MuO7boUdSc8,10.0.1.6,138.0,10.0.1.255,138.0,udp,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,conn,bobs.bigwheel.local,zeek,1.588205e+09,CuRbE21APSQo2qd6rk,10.0.1.6,123.0,10.0.0.4,123.0,udp,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
columnsToChangeType_day1= zeekDf_day1.select_dtypes(include=[object]).columns.to_list()
dictionary_day1 = {i : 'str' for i in columnsToChangeType_day1}
zeekDf_day1 = zeekDf_day1.astype(dictionary_day1)

In [20]:
df_zeek = spark.createDataFrame(zeekDf_day1)

print(type(df_zeek))
df_zeek.select('id_orig_h','id_orig_p','id_resp_h','id_resp_p').show(5, truncate = False)

<class 'pyspark.sql.dataframe.DataFrame'>
+---------+---------+----------+---------+
|id_orig_h|id_orig_p|id_resp_h |id_resp_p|
+---------+---------+----------+---------+
|10.0.1.6 |54243.0  |10.0.0.4  |53.0     |
|10.0.1.6 |56880.0  |10.0.0.4  |445.0    |
|10.0.1.6 |65108.0  |10.0.0.4  |53.0     |
|10.0.1.6 |138.0    |10.0.1.255|138.0    |
|10.0.1.6 |123.0    |10.0.0.4  |123.0    |
+---------+---------+----------+---------+
only showing top 5 rows



In [21]:
df_zeek.createTempView("apt29NetworkDay1")